In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time
import copy
import random
from collections import Counter, deque
from shapely.geometry import Point, MultiPoint
from shapely.geometry.polygon import Polygon
import ray
env = gym.make("Ant-v5")

#ray.init()

class Policy(nn.Module):
  #torch.manual_seed(191)
  def __init__(self):
      super(Policy, self).__init__()
      self.affine1 = nn.Linear(105,27)
      #self.affine2 = nn.Linear(32,32)
      self.action_head = nn.Linear(27,8)
      self.weights_initialization()
      self.setnogradient()
  def setnogradient(self):
    for param in self.parameters():
        param.requires_grad = False  
  def forward(self, inputs):
      x = inputs
      x = F.relu(self.affine1(x))
      #x = F.relu(self.affine2(x))
      return F.tanh(self.action_head(x))
  def weights_initialization(self):
      for m in self.modules():
          #print(m)
          #torch.manual_seed(191)
          if isinstance(m, nn.Linear):
              nn.init.xavier_normal_(m.weight)
              nn.init.constant_(m.bias, 0)
  def shapes_per_layer(self):
      return [list(p.data.shape) for p in self.parameters()]
  def count_parameters(self):
      return sum(p.numel() for p in self.parameters())

niche_real_counter = Counter()

@ray.remote
def get_bc_policy(policy):
    #global INDIVIDUAL_COUNTER
    #INDIVIDUAL_COUNTER += 1
    observation, info = env.reset(seed=42)
    for _ in range(1000):
        #action = env.action_space.sample()
        #print(type(observation))
        observation = torch.tensor(observation, dtype=torch.float32)
        action = policy(observation).detach().numpy()
        if pseudo:
            seeds = np.random.randint(0,3,3)
            action[seeds[0]] = 0
            action[seeds[1]] = 0
            action[seeds[2]] = 0
        #print(action)
        observation, reward, terminated, truncated, info = env.step(action)
        #if terminated or truncated:
    bc1 = observation["achieved_goal"][0]
    bc2 = observation["achieved_goal"][1]
    bc = np.array([bc1, bc2])
    #if random.random() < 0.95:
    archivebc.append(bc)
    return bc

def bc_a_set_of_policies(policies):
  return ray.get([get_bc_policy.remote(i) for i in policies])

dist = lambda p1, p2: np.sqrt(((p1-p2)**2).sum())

def get_fitnessess(xy_list, K=15):
   dm = np.asarray([[dist(p1, p2) for p2 in xy_list+archivebc] for p1 in xy_list+archivebc])
   dm.sort(axis=1)
   #print(dm)
   #print(dm[:,:(K+1)].sum(axis=1))
   return dm[:(POPULATION_SIZE+N_OF_OFFSPRING),:(K+1)].sum(axis=1)

def get_distance_from_parent_fitnesses(bcs):
    return [np.linalg.norm(bcs[i]-bcs[0]) for i in range(0,len(bcs))]

def mutate(agent):
    child_agent = copy.deepcopy(agent)
    mutation_power = 1
    for param in child_agent.parameters():
        if(len(param.shape)==2):
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    if random.random() > 0:
                        param[i0][i1]+= mutation_power * np.random.normal(0.1)
        elif(len(param.shape)==1):
            for i0 in range(param.shape[0]):
                if random.random() > 0:
                    param[i0]+=mutation_power * np.random.normal(0.1)
    return child_agent

POPULATION_SIZE = 1
N_OF_OFFSPRING = 200
archive_genotype = []
archivebc = []

policy = Policy()
print(policy.count_parameters())

def main():
   N_GENERATIONS = 1
   policies = [Policy() for i in range(POPULATION_SIZE)]
   for generation in range(N_GENERATIONS):
       #parent_bc = get_bc_policy.remote(policies[0])
       #print([parent_bc],11) 
       offspring = [mutate(random.choice(policies)) for i in range(N_OF_OFFSPRING)] 
       allmembers = policies+offspring
       #print(len(allmembers))
       bcs = bc_a_set_of_policies(allmembers)
       #bc_a_set_of_policies(allmembers, False)
       #print(bcs) 
       #for i in bcs:
        #    niche_real_number = int(np.floor(i)[0]*10+np.floor(i)[1])
        #    niche_real_counter.update([niche_real_number])
       #bc_a_set_of_policies(policies, False)
       #print(niche_real_counter)
       #print(len(niche_real_counter))
       #pseudobcs = bc_a_set_of_policies(allmembers, True)
       #print("Niches visited: {}".format(len(niche_counter)))
       #print(pseudo_bcs)
       #fitnesses = get_fitnessess(bcs)
       fitnesses = get_distance_from_parent_fitnesses(bcs)
       print(fitnesses)
       #print(fitnesses)
       surviving_indices = np.argsort(fitnesses)[::-1][:POPULATION_SIZE]
       print(surviving_indices)
       #print(surviving_indices)
       policies = [allmembers[i] for i in surviving_indices]
       #fitnesses = test_a_set_of_policies(policies)
       #print(fitnesses)
       #print(max(fitnesses))
       #print(archive)
       #print(len(archive))
       #print(archive_genotype)
       #print(len(archive))
       #print([archive_genotype[i] for i in np.random.randint(0,len(archive_genotype),POPULATION_SIZE//2)])
       #archivedbc = []
       #print(parent_indices)
       #parent_genotypes_post = [crossover(parent_genotypes[np.random.randint(POPULATION_SIZE)],parent_genotypes[np.random.randint(POPULATION_SIZE)]) for i in range(POPULATION_SIZE)]
       #elite_policy = copy.deepcopy(policies[parent_indices[0]])
       #print([policies[i].get_genotype() for i in parent_indices])
       #policies = generate_N_initial_policies(POPULATION_SIZE, 0.5, parent_genotypes)
       #policies[0] = elite_policy
       #print(np.all(parent_genotypes[6]==parent_genotypes[9]))
       #print("SVE OK")
       print("End of generation {}".format(generation+1))
   #np.save("bcs_xavier_zeros_shallow_2501", bcs)
   env.close()

main()



3086


RayTaskError(NameError): [36mray::get_bc_policy()[39m (pid=10160, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1879, in ray._raylet.execute_task
  File "C:\Users\bgasp\AppData\Local\Temp\ipykernel_4744\2017535369.py", line 58, in get_bc_policy
NameError: name 'pseudo' is not defined

2025-02-17 16:20:58,160	ERROR worker.py:422 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::get_bc_policy() (pid=27988, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1879, in ray._raylet.execute_task
  File "C:\Users\bgasp\AppData\Local\Temp\ipykernel_4744\2017535369.py", line 58, in get_bc_policy
NameError: name 'pseudo' is not defined
2025-02-17 16:20:58,160	ERROR worker.py:422 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::get_bc_policy() (pid=10160, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 1879, in ray._raylet.execute_task
  File "C:\Users\bgasp\AppData\Local\Temp\ipykernel_4744\2017535369.py", line 58, in get_bc_policy
NameError: name 'pseudo' is not defined
